# **nanoMPC** Data Maker Lite

This notebook connects to an app for augmenting a small dataset of single midi tracks into mixed drum beats. By inputting a small amount of data, you can generate a large number of variations.

To use this notebook, install the dependencies and upload midi files (Type 0) into the correct drum folders in `gm_drums_dataset`. Adjust the settings in 'Generate MIDIs' and then run the upload and download scripts. The augmented files will be unpacked in the `midi_output` directory, or you can download the file download.zip.

You can audition the midi files by uploading soundfont kits (.sf2) and running "Preview" to render an audio file. Please note the soundfont kit must be set to program 1 (GM Instrument - Acoustic Grand). The "Preview" mode is best for regenerating new batches and quickly saving your favorite generations.

---

nanoMPC Instrument Group Key Map (note numbers):

- Kick [36]
- Snare [37, 38]
- Clap [39]
- Hat [42, 44, 46]
- Tom [41, 43]
- Cymbals [49, 51]

The following notes will be remapped to a basic drum kit.

- 35 > 36 # Acoustic Bass Drum -> Bass Drum 1
- 40 > 38 # Electric Snare -> Acoustic Snare
- 52 > 49 # Chinese Cymbal -> Crash Cymbal 1
- 53 > 51 # Ride Bell -> Ride Cymbal 1
- 55 > 49 # Splash Cymbal -> Crash Cymbal 1
- 57 > 49 # Crash Cymbal 2 -> Crash Cymbal 1
- 59 > 51 # Ride Cymbal 2 -> Ride Cymbal 1

You can exclude notes by leaving a drum folder empty.

---
## For Best Results

- Midi files must be edited to 4 bars.
- Write your midi files as Instrument Groups (Example: For a hi-hat composed with notes 42, 44, and 46, write the file as a single track).
- Ensure the first and last notes do not extend beyond a 4-bar range.
- Focus on one drum style with the same timing or swing (avoid mixing straight and swing patterns unless desired).
- A larger dataset will yield more creative outputs. The more midi files you input, the more combinations become possible.
- The total upload size limit for all midi files in `gm_drums_dataset` is 1MB.
- The `num_samples` setting is capped at 100 mixes per run. You can regenerate unlimited batches.
- Velocities will be processed with a mastering script between the range of 40-100.

If you are interested in large-scale data augmentation, please contact us at aidev@patchbanks.com.

---
Disclaimer: We collect data on our servers for research and development purposes. By using this notebook, you agree to these terms and grant us the right to use your data. Please only upload midi files that you have permission to use.

This notebook is not fully tested and has limited app resources.

Patchbanks © 2024


# Install

In [ ]:
!apt-get install -q fluidsynth
!pip install -q pretty_midi

In [ ]:
from IPython.display import HTML, display
import os
import zipfile
import requests
import json
import shutil
from re import M
import base64
import subprocess
import random

In [ ]:
base_dir = '/content/'
midi_dataset = 'gm_drums_dataset'

drum_groups = [
    'Kick',
    'Snare',
    'Clap',
    'Hat',
    'Tom'
]


input_dir = os.path.join(base_dir, midi_dataset)
for group in drum_groups:
    group_dir = os.path.join(input_dir, group)
    os.makedirs(group_dir, exist_ok=True)


def create_dir(directory):
    os.makedirs(directory, exist_ok=True)

directories = [
    '/content/sf2_kits',
    '/content/midi_output',
    '/content/midi_saved'
]

for directory in directories:
    create_dir(directory)

print("Directories created successfully.")

In [ ]:
'''sf2_zip_url = 'https://github.com/user/repo/raw/branch/path/to/file.zip'

!wget -O /content/sf2_kits.zip {sf2_zip_url}

sf2_kits_dir = '/content/sf2_kits'
os.makedirs(sf2_kits_dir, exist_ok=True)

with zipfile.ZipFile('/content/sf2_kits.zip', 'r') as zip_ref:
    zip_ref.extractall(sf2_kits_dir)

!ls -l {sf2_kits_dir}'''

# Generate MIDIs

In [ ]:
#@title Upload

'''drum_style: tag your dataset with a genre or keyword'''

num_samples = 25 # @param {type: "slider", min: 1, max: 99}
set_bpm = 120 # @param {type: "number"}
user_id = 'guest_user' # do not change
drum_style = "pop" # @param {type: "string"}

user_data = {
    'num_samples': num_samples,
    'set_bpm': set_bpm,
    'user_id': user_id,
    'drum_style': drum_style
}


json_config = 'guest_user.json'
with open(json_config, 'w') as json_file:
    json.dump(user_data, json_file, indent=2)


user_dir = "/content/gm_drums_dataset"
zip_name = 'guest_user.zip'


def zip_midi_data(user_dir, output_zip):
    total_size = 0
    exceeded_limit = False
    with zipfile.ZipFile(output_zip, 'w') as zipf:
        for root, _, files in os.walk(user_dir):
            for file in files:
                if file.endswith('.mid'):
                    file_path = os.path.join(root, file)
                    arcname = os.path.relpath(file_path, user_dir)
                    file_size = os.path.getsize(file_path)
                    if total_size + file_size > 1024 * 1024:
                        exceeded_limit = True
                        break
                    zipf.write(file_path, arcname)
                    total_size += file_size
            if exceeded_limit:
                break

            for dir in os.listdir(root):
                dir_path = os.path.join(root, dir)
                if os.path.isdir(dir_path) and not dir.startswith('.'):
                    dir_size = sum(os.path.getsize(os.path.join(dir_path, f)) for f in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, f)))
                    if total_size + dir_size > 1024 * 1024:
                        exceeded_limit = True
                        break
                    zipf.write(dir_path, os.path.relpath(dir_path, user_dir))
                    total_size += dir_size
            if exceeded_limit:
                break

    if exceeded_limit:
        print("max upload size exceeded")
        return None

    print(f"upload file size: {total_size / (1024 * 1024):.2f} MB")
    return output_zip


zip_path = zip_midi_data(user_dir, zip_name)


if zip_path:
    upload_settings = 'http://patchbanks.pythonanywhere.com/upload_json'
    files = {'file': open(json_config, 'rb')}
    response_json = requests.post(upload_settings, files=files)

    if response_json.status_code == 200:
        print(f"user settings sent")
    else:
        print(f"error with status code {response_json.status_code}")

    upload_data = 'http://patchbanks.pythonanywhere.com/upload_zip'
    files = {'file': open(zip_path, 'rb')}
    response_zip = requests.post(upload_data, files=files)

    if response_zip.status_code == 200:
        print(f"midi data sent")
    else:
        print(f"error with status code {response_zip.status_code}")

    generate_midi = 'http://patchbanks.pythonanywhere.com/generate'
    generate_data = {
        'user_id': user_id,
        'num_samples': num_samples,
        'set_bpm': set_bpm
    }
    response_generate = requests.post(generate_midi, json=generate_data)

    if response_generate.status_code == 200:
        print(f"midi generation completed")
    else:
        print(f"error with status code {response_generate.status_code}")

In [ ]:
#@title Download

'''download midi generations to midi_output'''

download_url = 'http://patchbanks.pythonanywhere.com/download'

response = requests.post(download_url)

if response.status_code == 200:
    zip_filename = 'download.zip'
    with open(zip_filename, 'wb') as f:
        f.write(response.content)

    print(f"completed: {zip_filename}")

    output_dir = 'midi_output'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(output_dir):
        if filename.endswith('.mid'):
            file_path = os.path.join(output_dir, filename)
            os.remove(file_path)

    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall(output_dir)

elif response.status_code == 405:
    print(f"{response.status_code}")
else:
    print(f"{response.status_code}")
    print(f"{response.text}")


# Preview

In [ ]:
#@title Play MIDI

# @markdown Requires soundfont drum kits uploaded to `sf2_kits`.

autoplay = True # @param {type:"boolean"}
autoplay_attr = 'autoplay' if autoplay else ''
random_kit = False # @param {type: "boolean"}
sf2_kit = '' # @param {type: "string"}

if random_kit:
    sf2_kits_dir = '/content/sf2_kits'
    sf2_kit_list = [f.replace('.sf2', '') for f in os.listdir(sf2_kits_dir) if f.endswith('.sf2')]
    if sf2_kit_list:
        sf2_kit = random.choice(sf2_kit_list)
    else:
        raise FileNotFoundError("No SF2 kits found in the directory.")

sf2_file = f'/content/sf2_kits/{sf2_kit}.sf2'
print(f'sf2 kit: {sf2_kit}')

audio_format = 's16'  # s16 (16bit) or s24 (24bit)
sample_rate = 44100
gain = '1.5'

midi_output_dir = '/content/midi_output'

midi_files = [os.path.join(midi_output_dir, f) for f in os.listdir(midi_output_dir) if f.endswith('.mid')]

if not midi_files:
    raise FileNotFoundError(f"no midi files found in directory: {midi_output_dir}")

midi_file = random.choice(midi_files)
print('playing', midi_file)
print()

def render_wav(midi_file, output_wav, sample_rate, audio_format, gain):
    with open(os.devnull, 'w') as devnull:
        command = [
            'fluidsynth', '-ni', sf2_file, midi_file, '-F', output_wav,
            '-r', str(sample_rate), '-o', f'audio.file.format={audio_format}', '-g', str(gain)
        ]
        subprocess.call(command, stdout=devnull, stderr=devnull)

output_wav = '/content/sample.wav'
render_wav(midi_file, output_wav, sample_rate, audio_format, gain)

with open(output_wav, 'rb') as f:
    wav_data = f.read()
    base64_wav = base64.b64encode(wav_data).decode('utf-8')

audio_html = f'<audio controls {autoplay_attr}><source src="data:audio/wav;base64,{base64_wav}" type="audio/wav"></audio>'
display(HTML(audio_html))


In [ ]:
#@title Save MIDI

''' save the best generations to midi_saved '''

midi_save_dir = '/content/midi_saved'
if not os.path.exists(midi_save_dir):
    os.makedirs(midi_save_dir)

destination_file = os.path.join(midi_save_dir, os.path.basename(midi_file))

if os.path.exists(destination_file):
    print("midi file already exists")
else:
    shutil.move(midi_file, destination_file)
    print("saved midi file")

# Tools

In [ ]:
#@title Zip Midi Files

def zip_midi(zip_dir, file_name):
    zip_filename = f"{zip_dir}.zip"
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(zip_dir):
            for file in files:
                if 'guest_user' in file:
                    new_file = file.replace('guest_user', file_name)
                    file_path = os.path.join(root, file)
                    new_file_path = os.path.join(root, new_file)
                    os.rename(file_path, new_file_path)
                    zipf.write(new_file_path, os.path.relpath(new_file_path, zip_dir))
                else:
                    file_path = os.path.join(root, file)
                    zipf.write(file_path, os.path.relpath(file_path, zip_dir))
    return zip_filename

zip_dir = '/content/midi_saved' # @param {type: "string"}
rename_midi = 'drums' # @param {type: "string"}

if not os.path.exists(zip_dir):
    os.makedirs(zip_dir)

zip_filename = zip_midi(zip_dir, rename_midi)

In [ ]:
#@title Import Midi Dataset

# @markdown Used for uploading larger datasets from a zip file into gm_drums_dataset. Please make sure to structure the instrument group folders correctly.

zip_file = '/content/default.zip' # @param {type: "string"}
destination_dir = '/content/gm_drums_dataset'

subdirs = [name for name in os.listdir(destination_dir) if os.path.isdir(os.path.join(destination_dir, name))]

temp_dir = '/content/temp_unzip'
if os.path.exists(temp_dir):
    shutil.rmtree(temp_dir)
os.makedirs(temp_dir)

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(temp_dir)

for root, dirs, files in os.walk(temp_dir):
    for file in files:
        if file.endswith('.mid'):
            source_file_path = os.path.join(root, file)
            subdir_name = os.path.basename(root)
            if subdir_name in subdirs:
                destination_path = os.path.join(destination_dir, subdir_name, file)
                shutil.move(source_file_path, destination_path)

shutil.rmtree(temp_dir)
print("midi files imported")

In [ ]:
#@title Delete MIDI Directory

clear_dir = '/content/midi_output' # @param {type: "string"}

for root, dirs, files in os.walk(clear_dir):
    for filename in files:
        if filename.endswith('.mid'):
            file_path = os.path.join(root, filename)
            os.remove(file_path)
            print(f"Deleted: {file_path}")

print("All MIDI files have been deleted.")

GM2 Drum Map Reference

- 27_D#0_High_Q
- 28_E0_Slap
- 29_F0_Scratch_Push
- 30_F#0_Scratch_Pull
- 31_G0_Sticks
- 32_G0_Square_Click
- 33_G0_Metronome_Click
- 33_G0_Metronome_Bell
- 35_B0_Acoustic_Bass_Drum
- 36_C1_Bass_Drum_1
- 37_C#1_Side_Stick
- 38_D1_Acoustic_Snare
- 39_D#1_Hand_Clap
- 40_E1_ElectricSnare
- 41_F1_Low_Floor_Tom
- 42_F#1_Closed_Hi_Hat
- 43_G1_High_Floor_Tom
- 44_G#1_Pedal_Hi_Hat
- 45_A1_Low_Tom
- 46_A#1_Open_Hi_Hat
- 47_B1_Low_Mid_Tom
- 48_C2_High_Mid_Tom
- 49_C#2_Crash_Cymbal_1
- 50_D2_High_Tom
- 51_D#2_Ride Cymbal_1
- 52_E2_Chinese_Cymbal
- 53_F2_Ride_Bell
- 54_F#2_Tambourine
- 55_G2_Splash_Cymbal
- 56_G#2_Cowbell
- 57_A2_Crash_Cymbal 2
- 58_A#2_Vibraslap
- 59_B2_Ride Cymbal_2
- 60_C3_High_Bongo
- 61_C#3_Low_Bongo
- 62_D3_Mute_High_Conga
- 63_D#3_Open_High_Conga
- 64_E3_Low_Conga
- 65_F3_High_Timbale
- 66_F#3_Low_Timbale
- 67_G3_High_Agogo
- 68_G#3_Low_Agogo
- 69_A3_Cabasa
- 70_A#3_Maracas
- 71_B3_Short_Whistle
- 72_C4_Long_Whistle
- 73_C#4_Short_Guiro
- 74_D4_Long_Guiro
- 75_D#4_Claves
- 76_E4_High_Wood_Block
- 77_F4_Low_Wood_Block
- 78_F#4_Mute_Cuica
- 79_G4_Open_Cuica
- 80_G#4_Mute_Triangle
- 81_A4_Open_Triangle
- 82_A#4_Shaker
- 83_B4_Jingle_Bell
- 84_C4_Belltree
- 85_C#4_Castanets
- 86_D4_Mute_Surdo
- 87_D#4_Open_Surdo